In [236]:
import pandas as pd
import numpy as np

In [237]:
PAN_data = pd.read_csv("PAN_data.csv")

C:\Users\Omar\AppData\Local\Temp\ipykernel_1452\381285905.py:1: DtypeWarning: Columns (3,6,9,18,19,20,21,22,23,24,28,29,30,31,32,34,35,36,37,38,39,45,46,47,48,49,52,53,54,55,66,67,70,72,74,75,76,77,78,79,82,83,84,85,86,87,88,89,90,91,92,93,94,103,104,106,112,115,116,117,120,122,123,132,134,135,136,138,140,141,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,169,171,173,175,176,177,178,179,180,181,182,183,184,188,189,190,191,193,199,200,202,203,204,207,208,209,210,211,212,213,214,216,217,222,223,224,225,226,227,228,234,236,237,242,243,248,253,255,260,261,262,263,264,266,267,268,269,270,271,277,280,281,283,285,287,289,290,294,295,296,297,299,300,301,302,303,306,308,317,321,328,330,332,347,352,357,358,365,366,367,368,372,379,382,386,387,388,393,410,411,412,413,414,421,422,429,430,431,432,433,434,435,436,438,442,455,456,457,458,459,468,469,470,471) have mixed types. Specify dtype option on import or set low_memory=False.
  PAN_data = pd.read_csv("PAN_data.csv")


In [238]:
after_94 = pd.to_datetime("04/01/1994", dayfirst=False)
pan_data_after_94 = PAN_data.copy()
pan_data_after_94["TX_DATE"] = pd.to_datetime(pan_data_after_94["TX_DATE"], dayfirst=False)
pan_data_after_94 = pan_data_after_94[pan_data_after_94["TX_DATE"] > after_94]

In [239]:
my_cols = [
    "GENDER", "PERIP_VASC", "AGE_DIAB", "ABO", "ABO_DON", "COD_CAD_DON", "DEATH_MECH_DON",
    "CREAT_TRR",
    "AMIS", "BMIS", "DRMIS", "HLAMIS", "NPKID", "NPPAN", "AGE_DON", "DDAVP_DON", "CMV_DON",
    "GENDER_DON",
    "NON_HRT_DON", "ANTIHYPE_DON", "BUN_DON", "CREAT_DON", 
    "PT_DIURETICS_DON", "PT_STEROIDS_DON", "PT_T4_DON", "SGOT_DON",
    "SGPT_DON", "TBILI_DON", "VASODIL_DON", "CLIN_INFECT_DON", "HIST_CIG_DON",
    "HIST_HYPERTENS_DON", "HIST_COCAINE_DON", "HIST_OTH_DRUG_DON", "HEPARIN_DON", 
    "BMI_DON_CALC", "ABO_MAT", "AGE", "DIAL_TRR", "ART_RECON", "DUCT_MGMT", "GRF_PLACEM",
    "PA_PRESERV_TM", "VASC_MGMT", "VEN_EXT_GRF",
    "DAYSWAIT_CHRON_PA", "EBV_SEROSTATUS", "HBV_CORE",
    "HCV_SEROSTATUS", "CMV_STATUS",
    "MALIG", "BMI_CALC",
    "PROTEIN_URINE", "LIPASE", "AMYLASE", "CARDARREST_NEURO", "INOTROP_SUPPORT_DON",   
    "REM_CD", "AGE_GROUP", "DON_TY", "MULTIORG", "TX_DATE","FAILDATE_PA","PX_STAT_DATE", "DIAB", "GRF_STAT_PA"
]


In [240]:
clean_good = pan_data_after_94[my_cols]

In [241]:
# convert the reason of removal to numeric and only include those that underwent transplantation [2,3,4,14,15,18,19]
clean_good["REM_CD"] = clean_good["REM_CD"].replace(to_replace=".", value=0)
clean_good["REM_CD"] = pd.to_numeric(clean_good["REM_CD"])

C:\Users\Omar\AppData\Local\Temp\ipykernel_1452\2907620572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_good["REM_CD"] = clean_good["REM_CD"].replace(to_replace=".", value=0)
C:\Users\Omar\AppData\Local\Temp\ipykernel_1452\2907620572.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_good["REM_CD"] = pd.to_numeric(clean_good["REM_CD"])


In [242]:
clean_good = clean_good[clean_good["REM_CD"].isin([2,3,4,14,15,18,19])]

In [243]:
#remove transplants with no followup

clean_good = clean_good[clean_good["PX_STAT_DATE"] != "."]
print(clean_good.shape)

(34024, 66)


In [244]:
clean_good["PX_STAT_DATE"] = pd.to_datetime(clean_good["PX_STAT_DATE"], dayfirst=False)
clean_good["TX_DATE"] = pd.to_datetime(clean_good["TX_DATE"], dayfirst=False)

In [245]:
very_clean_data = clean_good.copy()

In [246]:
very_clean_data = very_clean_data[very_clean_data["AGE_GROUP"] == "A"]

very_clean_data = very_clean_data[very_clean_data["DON_TY"] == "C"]

In [247]:
data_pre_imp = very_clean_data.copy()

In [248]:
data_pre_imp = data_pre_imp[data_pre_imp["HLAMIS"] != "."]

In [249]:
#grafts that survived five years
surv = data_pre_imp[data_pre_imp["GRF_STAT_PA"] == "Y"]
print(surv.shape)
surv = surv[surv["FAILDATE_PA"] == "."]
print(surv.shape)
surv["time_frame"] = surv["PX_STAT_DATE"] - surv["TX_DATE"]
surv["time_frame"] = surv["time_frame"].dt.days


(23436, 66)
(23431, 66)


In [250]:

fail = data_pre_imp[data_pre_imp["GRF_STAT_PA"] == "N"]
print(fail.shape)
#drop data wihtout fail date
fail = fail[fail["FAILDATE_PA"] != "."]
print(fail.shape)
fail["FAILDATE_PA"] = pd.to_datetime(fail["FAILDATE_PA"], dayfirst=False)
print(fail.shape)
fail["fail_frame"] = fail["FAILDATE_PA"] - fail["TX_DATE"] 
print(fail.shape)
fail["fail_frame"] = fail["fail_frame"].dt.days
print(fail.shape)
fail["time_frame"] = fail["fail_frame"]
fail = fail[fail["time_frame"] >= 0] 
fail = fail.drop(columns=["fail_frame"], axis=1)
print(fail.shape)


(9568, 66)
(9559, 66)
(9559, 66)
(9559, 67)
(9559, 67)
(9534, 67)


In [251]:
frames = [surv, fail]
labelled_pre_imp = pd.concat(frames)


In [252]:
# labelled_pre_imp = labelled_pre_imp.sort_values(['index'])
no_longer_needed = ["PX_STAT_DATE", "FAILDATE_PA", 
                    "TX_DATE", "AGE_GROUP", "DON_TY", "REM_CD"]
labelled_pre_imp = labelled_pre_imp.drop(columns=no_longer_needed)

In [253]:
# remove 86 donors with unkown height or weight or both
labelled_pre_imp = labelled_pre_imp[labelled_pre_imp["BMI_DON_CALC"] != "."]

#remove recipients with unkown height weight or both
labelled_pre_imp = labelled_pre_imp[labelled_pre_imp["BMI_CALC"] != "."]

In [254]:
imputation = labelled_pre_imp.copy()

In [255]:
imputation = imputation.replace(to_replace="U", value=np.nan)

In [256]:

def clean_data(imputation):
    # Replace all instances of "" with "" in columns: 'GENDER', 'GENDER_DON'
    imputation['GENDER'] = imputation['GENDER'].str.replace("M", "0", case=False, regex=False)
    imputation['GENDER_DON'] = imputation['GENDER_DON'].str.replace("M", "0", case=False, regex=False)
    imputation['GENDER'] = imputation['GENDER'].str.replace("f", "1", case=False, regex=False)
    imputation['GENDER_DON'] = imputation['GENDER_DON'].str.replace("F", "1", case=False, regex=False)
    # Replace missing values with "N" in column: 'PERIP_VASC' : the most common value
    imputation = imputation.fillna({'PERIP_VASC': "N"})
    # Replace all instances of "." with "" in column: 'AGE_DIAB'
    imputation.loc[imputation['AGE_DIAB'].str.lower() == ".".lower(), 'AGE_DIAB'] = np.nan
    # Change column type to float32 for column: 'AGE_DIAB'
    imputation = imputation.astype({'AGE_DIAB': 'float32'})
    # Replace missing values with the median of each column in: 'AGE_DIAB'
    imputation = imputation.fillna({'AGE_DIAB': imputation['AGE_DIAB'].median()})
    # Change column type to string for columns: 'ABO', 'ABO_DON' and 3 other columns
    imputation = imputation.astype({'ABO': 'string', 'ABO_DON': 'string', 'COD_CAD_DON': 'string', 'DEATH_MECH_DON': 'string'})
    # Replace all instances of "." with "" in column: 'CREAT_TRR'
    imputation.loc[imputation['CREAT_TRR'].str.lower() == ".".lower(), 'CREAT_TRR'] = np.nan
    # Change column type to float32 for column: 'CREAT_TRR'
    imputation = imputation.astype({'CREAT_TRR': 'float32'})
    # Replace missing values with the median of each column in: 'CREAT_TRR'
    imputation = imputation.fillna({'CREAT_TRR': imputation['CREAT_TRR'].median()})
    # Change column type to float32 for columns: 'AMIS', 'BMIS' and 2 other columns
    imputation = imputation.astype({'AMIS': 'float32', 'BMIS': 'float32', 'DRMIS': 'float32', 'HLAMIS': 'float32'})
    # One-hot encode columns: 'ABO', 'ABO_DON'
    for column in ['ABO', 'ABO_DON']:
        insert_loc = imputation.columns.get_loc(column)
        imputation = pd.concat([imputation.iloc[:,:insert_loc], pd.get_dummies(imputation.loc[:, [column]]), imputation.iloc[:,insert_loc+1:]], axis=1)
    # Replace all instances of "4" with "" in column: 'COD_CAD_DON'
    imputation['COD_CAD_DON'] = imputation['COD_CAD_DON'].str.replace("4", "other", case=False, regex=False)
    imputation['COD_CAD_DON'] = imputation['COD_CAD_DON'].str.replace("999", "other", case=False, regex=False)
    imputation['COD_CAD_DON'] = imputation['COD_CAD_DON'].str.replace(".", "other", case=False, regex=False)
    # One-hot encode column: 'COD_CAD_DON'
    insert_loc = imputation.columns.get_loc('COD_CAD_DON')
    imputation = pd.concat([imputation.iloc[:,:insert_loc], pd.get_dummies(imputation.loc[:, ['COD_CAD_DON']]), imputation.iloc[:,insert_loc+1:]], axis=1)
    # Rename column 'COD_CAD_DON_1' to 'COD_CAD_DON_anoxia'
    imputation = imputation.rename(columns={'COD_CAD_DON_1': 'COD_CAD_DON_anoxia'})
    # Rename column 'COD_CAD_DON_2' to 'COD_CAD_DON_stroke'
    imputation = imputation.rename(columns={'COD_CAD_DON_2': 'COD_CAD_DON_stroke'})
    # Rename column 'COD_CAD_DON_3' to 'COD_CAD_DON_headTrauma'
    imputation = imputation.rename(columns={'COD_CAD_DON_3': 'COD_CAD_DON_headTrauma'})
    # Replace all instances of "5" with "other" in column: 'DEATH_MECH_DON'
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "5".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "10".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "3".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "12".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "4".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "6".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "995".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "997".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "2".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "1".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == "8".lower(), 'DEATH_MECH_DON'] = "other"
    imputation.loc[imputation['DEATH_MECH_DON'].str.lower() == ".".lower(), 'DEATH_MECH_DON'] = "other"
    # One-hot encode column: 'DEATH_MECH_DON'
    insert_loc = imputation.columns.get_loc('DEATH_MECH_DON')
    imputation = pd.concat([imputation.iloc[:,:insert_loc], pd.get_dummies(imputation.loc[:, ['DEATH_MECH_DON']]), imputation.iloc[:,insert_loc+1:]], axis=1)
    # Rename column 'DEATH_MECH_DON_11' to 'DEATH_MECH_DON_ICH'
    imputation = imputation.rename(columns={'DEATH_MECH_DON_11': 'DEATH_MECH_DON_ICH'})
    # Rename column 'DEATH_MECH_DON_7' to 'DEATH_MECH_DON_GSW'
    imputation = imputation.rename(columns={'DEATH_MECH_DON_7': 'DEATH_MECH_DON_GSW'})
    # Rename column 'DEATH_MECH_DON_9' to 'DEATH_MECH_DON_bluntInjury'
    imputation = imputation.rename(columns={'DEATH_MECH_DON_9': 'DEATH_MECH_DON_bluntInjury'})
    # Change column type to float32 for column: 'AGE_DON'
    imputation = imputation.astype({'AGE_DON': 'float32'})
    # Replace missing values with "N" in column: 'DDAVP_DON': most common value
    imputation = imputation.fillna({'DDAVP_DON': "N"})
    # Replace all instances of "ND" with "N" in column: 'CMV_DON'
    imputation.loc[imputation['CMV_DON'].str.lower() == "ND".lower(), 'CMV_DON'] = "N"
    imputation.loc[imputation['CMV_DON'].str.lower() == "I".lower(), 'CMV_DON'] = "N"
    imputation.loc[imputation['CMV_DON'].str.lower() == "C".lower(), 'CMV_DON'] = "N"
    # Replace missing values with "N" in column: 'CMV_DON'
    imputation = imputation.fillna({'CMV_DON': "N"})
    # Replace missing values with "N" in columns: 'NON_HRT_DON', 'ANTIHYPE_DON'
    imputation = imputation.fillna({'NON_HRT_DON': "N", 'ANTIHYPE_DON': "N"})
    # Change column type to string for columns: 'BUN_DON', 'CREAT_DON' and 3 other columns
    imputation = imputation.astype({'BUN_DON': 'string', 'CREAT_DON': 'string', 'SGOT_DON': 'string', 'SGPT_DON': 'string', 'TBILI_DON': 'string'})
    # Replace all instances of "." with "" in column: 'BUN_DON'
    imputation.loc[imputation['BUN_DON'].str.lower() == ".".lower(), 'BUN_DON'] = np.nan
    imputation.loc[imputation['CREAT_DON'].str.lower() == ".".lower(), 'CREAT_DON'] = np.nan
    # Change column type to float32 for columns: 'CREAT_DON', 'BUN_DON'
    imputation = imputation.astype({'CREAT_DON': 'float32', 'BUN_DON': 'float32'})
    # Replace missing values with the median of each column in: 'BUN_DON', 'CREAT_DON'
    imputation = imputation.fillna({'BUN_DON': imputation['BUN_DON'].median(), 'CREAT_DON': imputation['CREAT_DON'].median()})
    # Replace missing values with "Y" in column: 'PT_DIURETICS_DON'
    imputation = imputation.fillna({'PT_DIURETICS_DON': "Y"})
    # Replace missing values with "Y" in column: 'PT_STEROIDS_DON'
    imputation = imputation.fillna({'PT_STEROIDS_DON': "Y"})
    # Replace missing values with "Y" in column: 'PT_T4_DON'
    imputation = imputation.fillna({'PT_T4_DON': "Y"})
    # Replace all instances of "." with "" in column: 'SGOT_DON'
    imputation.loc[imputation['SGOT_DON'].str.lower() == ".".lower(), 'SGOT_DON'] = np.nan
    imputation.loc[imputation['SGPT_DON'].str.lower() == ".".lower(), 'SGPT_DON'] = np.nan
    imputation.loc[imputation['TBILI_DON'].str.lower() == ".".lower(), 'TBILI_DON'] = np.nan
    # Change column type to float32 for columns: 'SGPT_DON', 'SGOT_DON', 'TBILI_DON'
    imputation = imputation.astype({'SGPT_DON': 'float32', 'SGOT_DON': 'float32', 'TBILI_DON': 'float32'})
    # Replace missing values with the median of each column in: 'SGOT_DON', 'SGPT_DON', 'TBILI_DON'
    imputation = imputation.fillna({'SGOT_DON': imputation['SGOT_DON'].median(), 'SGPT_DON': imputation['SGPT_DON'].median(), 'TBILI_DON': imputation['TBILI_DON'].median()})
    # Replace missing values with "N" in column: 'VASODIL_DON'
    imputation = imputation.fillna({'VASODIL_DON': "N"})
    # Replace missing values with "N" in column: 'CLIN_INFECT_DON'
    imputation = imputation.fillna({'CLIN_INFECT_DON': "N"})
    # Replace missing values with "N" in column: 'HIST_CIG_DON'
    imputation = imputation.fillna({'HIST_CIG_DON': "N"})
    # Replace missing values with "N" in column: 'HIST_HYPERTENS_DON'
    imputation = imputation.fillna({'HIST_HYPERTENS_DON': "N"})
    # Replace missing values with "N" in column: 'HIST_COCAINE_DON'
    imputation = imputation.fillna({'HIST_COCAINE_DON': "N"})
    # Replace missing values with "N" in column: 'HIST_OTH_DRUG_DON'
    imputation = imputation.fillna({'HIST_OTH_DRUG_DON': "N"})
    # Replace missing values with "Y" in column: 'HEPARIN_DON'
    imputation = imputation.fillna({'HEPARIN_DON': "Y"})
    # Replace missing values with "Y" in column: 'DIAL_TRR'
    imputation = imputation.fillna({'DIAL_TRR': "Y"})
    # Change column type to string for columns: 'VASC_MGMT', 'GRF_PLACEM' and 2 other columns
    imputation = imputation.astype({'VASC_MGMT': 'string', 'GRF_PLACEM': 'string', 'DUCT_MGMT': 'string', 'ART_RECON': 'string'})
    # Replace all instances of "3" with "" in column: 'ART_RECON'
    imputation.loc[imputation['ART_RECON'].str.lower() == "3".lower(), 'ART_RECON'] = "0"
    imputation.loc[imputation['ART_RECON'].str.lower() == "999".lower(), 'ART_RECON'] = "0"
    imputation.loc[imputation['ART_RECON'].str.lower() == "5".lower(), 'ART_RECON'] = "0"
    imputation.loc[imputation['ART_RECON'].str.lower() == "4".lower(), 'ART_RECON'] = "0"
    imputation.loc[imputation['ART_RECON'].str.lower() == "1".lower(), 'ART_RECON'] = "0"
    imputation.loc[imputation['ART_RECON'].str.lower() == ".".lower(), 'ART_RECON'] = "0"
    imputation.loc[imputation['ART_RECON'].str.lower() == "2".lower(), 'ART_RECON'] = "1"
    # Rename column 'ART_RECON' to 'ART_RECON_Y-Graft'
    imputation = imputation.rename(columns={'ART_RECON': 'ART_RECON_Y-Graft'})
    # Replace all instances of "999" with "other" in column: 'DUCT_MGMT'
    imputation.loc[imputation['DUCT_MGMT'].str.lower() == "999".lower(), 'DUCT_MGMT'] = "other"
    imputation.loc[imputation['DUCT_MGMT'].str.lower() == "4".lower(), 'DUCT_MGMT'] = "other"
    imputation.loc[imputation['DUCT_MGMT'].str.lower() == "5".lower(), 'DUCT_MGMT'] = "other"
    imputation.loc[imputation['DUCT_MGMT'].str.lower() == ".".lower(), 'DUCT_MGMT'] = "other"
    # One-hot encode column: 'DUCT_MGMT'
    insert_loc = imputation.columns.get_loc('DUCT_MGMT')
    imputation = pd.concat([imputation.iloc[:,:insert_loc], pd.get_dummies(imputation.loc[:, ['DUCT_MGMT']]), imputation.iloc[:,insert_loc+1:]], axis=1)
    # Rename column 'DUCT_MGMT_1' to 'DUCT_MGMT_ENTERIC W/ROUX-EN-Y'
    imputation = imputation.rename(columns={'DUCT_MGMT_1': 'DUCT_MGMT_ENTERIC W/ROUX-EN-Y'})
    # Rename column 'DUCT_MGMT_2' to 'DUCT_MGMT_ENTERIC W/O ROUX-EN-Y'
    imputation = imputation.rename(columns={'DUCT_MGMT_2': 'DUCT_MGMT_ENTERIC W/O ROUX-EN-Y'})
    # Rename column 'DUCT_MGMT_3' to 'DUCT_MGMT_cystostomy'
    imputation = imputation.rename(columns={'DUCT_MGMT_3': 'DUCT_MGMT_cystostomy'})
    # Replace all instances of "2" with "0" in column: 'GRF_PLACEM'
    imputation.loc[imputation['GRF_PLACEM'].str.lower() == "2".lower(), 'GRF_PLACEM'] = "0"
    imputation.loc[imputation['GRF_PLACEM'].str.lower() == "3".lower(), 'GRF_PLACEM'] = "0"
    imputation.loc[imputation['GRF_PLACEM'].str.lower() == ".".lower(), 'GRF_PLACEM'] = "0"
    # Rename column 'GRF_PLACEM' to 'GRF_PLACEM_intra'
    imputation = imputation.rename(columns={'GRF_PLACEM': 'GRF_PLACEM_intra'})
    # Change column type to string for column: 'PA_PRESERV_TM'
    imputation = imputation.astype({'PA_PRESERV_TM': 'string'})
    # Replace all instances of "." with "" in column: 'PA_PRESERV_TM'
    imputation.loc[imputation['PA_PRESERV_TM'].str.lower() == ".".lower(), 'PA_PRESERV_TM'] = np.nan
    # Change column type to float32 for column: 'PA_PRESERV_TM'
    imputation = imputation.astype({'PA_PRESERV_TM': 'float32'})
    # Replace missing values with the median of each column in: 'PA_PRESERV_TM'
    imputation = imputation.fillna({'PA_PRESERV_TM': imputation['PA_PRESERV_TM'].median()})
    # Replace all instances of "." with "1" in column: 'VASC_MGMT'
    imputation['VASC_MGMT'] = imputation['VASC_MGMT'].str.replace(".", "1", case=False, regex=False)
    # One-hot encode column: 'VASC_MGMT'
    insert_loc = imputation.columns.get_loc('VASC_MGMT')
    imputation = pd.concat([imputation.iloc[:,:insert_loc], pd.get_dummies(imputation.loc[:, ['VASC_MGMT']]), imputation.iloc[:,insert_loc+1:]], axis=1)
    # Rename column 'VASC_MGMT_1' to 'VASC_MGMT_systemic'
    imputation = imputation.rename(columns={'VASC_MGMT_1': 'VASC_MGMT_systemic'})
    # Rename column 'VASC_MGMT_2' to 'VASC_MGMT_portal'
    imputation = imputation.rename(columns={'VASC_MGMT_2': 'VASC_MGMT_portal'})
    # Rename column 'VASC_MGMT_3' to 'VASC_MGMT_na/multiOrg'
    imputation = imputation.rename(columns={'VASC_MGMT_3': 'VASC_MGMT_na/multiOrg'})
    # Replace missing values with "N" in column: 'VEN_EXT_GRF'
    imputation = imputation.fillna({'VEN_EXT_GRF': "N"})
    # Change column type to string for column: 'DAYSWAIT_CHRON_PA'
    imputation = imputation.astype({'DAYSWAIT_CHRON_PA': 'string'})
    # Replace all instances of "." with "" in column: 'DAYSWAIT_CHRON_PA'
    imputation.loc[imputation['DAYSWAIT_CHRON_PA'].str.lower() == ".".lower(), 'DAYSWAIT_CHRON_PA'] = np.nan
    # Change column type to float32 for column: 'DAYSWAIT_CHRON_PA'
    imputation = imputation.astype({'DAYSWAIT_CHRON_PA': 'float32'})
    # Replace missing values with the median of each column in: 'DAYSWAIT_CHRON_PA'
    imputation = imputation.fillna({'DAYSWAIT_CHRON_PA': imputation['DAYSWAIT_CHRON_PA'].median()})
    # Replace all instances of "ND" with "N" in columns: 'HBV_CORE', 'EBV_SEROSTATUS' and 2 other columns
    imputation['HBV_CORE'] = imputation['HBV_CORE'].str.replace("ND", "N", case=False, regex=False)
    imputation['EBV_SEROSTATUS'] = imputation['EBV_SEROSTATUS'].str.replace("ND", "N", case=False, regex=False)
    imputation['HCV_SEROSTATUS'] = imputation['HCV_SEROSTATUS'].str.replace("ND", "N", case=False, regex=False)
    imputation['CMV_STATUS'] = imputation['CMV_STATUS'].str.replace("ND", "N", case=False, regex=False)
    # Replace missing values with "P" in columns: 'EBV_SEROSTATUS', 'CMV_STATUS'
    imputation = imputation.fillna({'EBV_SEROSTATUS': "P", 'CMV_STATUS': "P"})
    # Replace missing values with "N" in columns: 'HCV_SEROSTATUS', 'HBV_CORE'
    imputation = imputation.fillna({'HCV_SEROSTATUS': "N", 'HBV_CORE': "N"})
    # Replace missing values with "N" in column: 'MALIG'
    imputation = imputation.fillna({'MALIG': "N"})
    # Replace missing values with "N" in column: 'PROTEIN_URINE'
    imputation = imputation.fillna({'PROTEIN_URINE': "N"})
    # Replace missing values with "N" in column: 'CARDARREST_NEURO'
    imputation = imputation.fillna({'CARDARREST_NEURO': "N"})
    # Replace missing values with "N" in column: 'INOTROP_SUPPORT_DON'
    imputation = imputation.fillna({'INOTROP_SUPPORT_DON': "N"})
    # Replace missing values with "N" in column: 'MULTIORG'
    imputation = imputation.fillna({'MULTIORG': "N"})
    # Replace all instances of "." with "" in columns: 'LIPASE', 'AMYLASE'
    imputation.loc[imputation['LIPASE'].str.lower() == ".".lower(), 'LIPASE'] = np.nan
    imputation.loc[imputation['AMYLASE'].str.lower() == ".".lower(), 'AMYLASE'] = np.nan
    # Change column type to float32 for columns: 'LIPASE', 'AMYLASE'
    imputation = imputation.astype({'LIPASE': 'float32', 'AMYLASE': 'float32'})
    # Replace missing values with the median of each column in: 'LIPASE', 'AMYLASE'
    imputation = imputation.fillna({'LIPASE': imputation['LIPASE'].median(), 'AMYLASE': imputation['AMYLASE'].median()})
    # Change column type to string for column: 'DIAB'
    imputation = imputation.astype({'DIAB': 'string'})
    # Replace all instances of "1" with "0" in column: 'DIAB'
    imputation.loc[imputation['DIAB'].str.lower() == "1".lower(), 'DIAB'] = "0"
    imputation.loc[imputation['DIAB'].str.lower() == "998".lower(), 'DIAB'] = "0"
    imputation.loc[imputation['DIAB'].str.lower() == ".".lower(), 'DIAB'] = "0"
    imputation.loc[imputation['DIAB'].str.lower() == "2".lower(), 'DIAB'] = "1"
    imputation.loc[imputation['DIAB'].str.lower() == "3".lower(), 'DIAB'] = "1"
    imputation.loc[imputation['DIAB'].str.lower() == "4".lower(), 'DIAB'] = "1"
    imputation.loc[imputation['DIAB'].str.lower() == "5".lower(), 'DIAB'] = "1"
    # Rename column 'DIAB' to 'DIAB_Y/N'
    imputation = imputation.rename(columns={'DIAB': 'DIAB_Y/N'})


    imputation = imputation.rename(columns={'GRF_STAT_PA': 'GRF_STAT_PA_didFail'})  

    imputation['GRF_STAT_PA_didFail'] = imputation['GRF_STAT_PA_didFail'].str.replace("Y", "0", case=False, regex=False)
    imputation['GRF_STAT_PA_didFail'] = imputation['GRF_STAT_PA_didFail'].str.replace("N", "1", case=False, regex=False)

    return imputation

imputation_clean = clean_data(imputation.copy())

In [257]:
imputation_clean = imputation_clean.replace(to_replace="N", value=0)
imputation_clean = imputation_clean.replace(to_replace="Y", value=1)
imputation_clean = imputation_clean.replace(to_replace="P", value=1)

C:\Users\Omar\AppData\Local\Temp\ipykernel_1452\430475599.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  imputation_clean = imputation_clean.replace(to_replace="Y", value=1)
C:\Users\Omar\AppData\Local\Temp\ipykernel_1452\430475599.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  imputation_clean = imputation_clean.replace(to_replace="P", value=1)


In [258]:
imputation_clean[imputation_clean.select_dtypes(include=['int']).columns] = imputation_clean.select_dtypes(include=['int']).astype("float32")

imputation_clean[imputation_clean.select_dtypes(include=['string']).columns] = imputation_clean.select_dtypes(include=['string']).astype("float32")

imputation_clean[imputation_clean.select_dtypes(include=['object']).columns] = imputation_clean.select_dtypes(include=['object']).astype("float32")

imputation_clean[imputation_clean.select_dtypes(include=['bool']).columns] = imputation_clean.select_dtypes(include=['bool']).astype("int")

In [259]:
imputation_clean["YRS_DIAB"] = (imputation_clean["AGE"] - imputation_clean["AGE_DIAB"]) * imputation_clean["DIAB_Y/N"]

imputation_clean["YRS_DIAB"] = imputation_clean["YRS_DIAB"].clip(lower=0)

imputation_clean = imputation_clean.drop("AGE_DIAB", axis=1)

In [260]:
skewed_cols = ["BUN_DON", "CREAT_DON", "SGOT_DON", "SGPT_DON", "TBILI_DON", "LIPASE", "AMYLASE"]

In [261]:
imputation_clean_winsor = imputation_clean.copy()

# Define the lower and upper percentiles
lower_percentile = 0.00
upper_percentile = 0.95

# Apply percentile-based capping
imputation_clean_winsor[skewed_cols] = imputation_clean_winsor[skewed_cols].clip(lower=imputation_clean_winsor[skewed_cols].quantile(lower_percentile), upper=imputation_clean_winsor[skewed_cols].quantile(upper_percentile), axis=1)



In [262]:
imputation_clean = imputation_clean_winsor.copy()

In [263]:
numeric_cols = [
    "CREAT_TRR", "AMIS", "BMIS", "DRMIS", "HLAMIS", "NPKID", "NPPAN", "AGE_DON", "BUN_DON", 
    "CREAT_DON", "SGOT_DON", "SGPT_DON", "TBILI_DON", "BMI_DON_CALC", "AGE", "PA_PRESERV_TM", 
    "DAYSWAIT_CHRON_PA", "BMI_CALC", "LIPASE", "AMYLASE", "YRS_DIAB"
]


In [264]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

scaler = MinMaxScaler()

scaled_df = imputation_clean.copy()

scaled_df[numeric_cols] = scaler.fit_transform(scaled_df[numeric_cols])



In [265]:
scaled_df.to_csv("data_ready_final.csv", index=False)